In [13]:
cd ..

D:\valkyrie\python


In [52]:
import requests
import urllib
from html_table_parser.parser import HTMLTableParser
from universe import *
from valkyrie_tools import *
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
import pandas as pd

In [66]:
df  = pd.read_hdf('D:\\valkyrie\\data\\hist_ib_raw_md\\2020\\11\\04\\bid_ask\\BAC PRB.h5')

In [68]:
df2 = pd.read_hdf('D:\\valkyrie\\data\\hist_ib_raw_md\\2020\\11\\04\\trades\\BAC PRB.h5')

In [69]:
df2.head()

,price,size,exchange,specialConditions,ticker
time,,,,,
2020-11-04 09:12:22-05:00,26.700,1,ARCA,TI,BAC PRB
2020-11-04 09:30:00-05:00,26.900,1005,NYSE,O,BAC PRB
2020-11-04 09:30:11-05:00,26.900,21,FINRA,4 I,BAC PRB
2020-11-04 09:31:15-05:00,26.915,1300,FINRA,,BAC PRB
2020-11-04 09:31:35-05:00,26.970,67,IEX,I,BAC PRB


In [67]:
df.head()

,priceBid,priceAsk,sizeBid,sizeAsk,ticker
time,,,,,
2020-11-04 04:00:00-05:00,26.39,0.00,100,0,BAC PRB
2020-11-04 07:00:27-05:00,26.39,27.97,100,5700,BAC PRB
2020-11-04 09:28:26-05:00,26.39,27.97,300,5900,BAC PRB
2020-11-04 09:29:45-05:00,26.39,27.97,400,5900,BAC PRB
2020-11-04 09:29:45-05:00,26.39,27.97,400,6000,BAC PRB


In [55]:
xx = 'D:\\valkyrie\\data\\hist_ib_raw_md\\2020\\11\\04\\bid_ask\\BAC PRB.h5'.replace('\\','//')

In [56]:
df_x = pd.read_hdf(xx)

In [57]:
df_x

,priceBid,priceAsk,sizeBid,sizeAsk,ticker
time,,,,,
2020-11-04 04:00:00-05:00,26.39,0.00,100,0,BAC PRB
2020-11-04 07:00:27-05:00,26.39,27.97,100,5700,BAC PRB
2020-11-04 09:28:26-05:00,26.39,27.97,300,5900,BAC PRB
2020-11-04 09:29:45-05:00,26.39,27.97,400,5900,BAC PRB
2020-11-04 09:29:45-05:00,26.39,27.97,400,6000,BAC PRB
...,...,...,...,...,...
2020-11-04 16:00:01-05:00,26.39,0.00,100,0,BAC PRB
2020-11-04 16:00:25-05:00,26.40,0.00,200,0,BAC PRB
2020-11-04 16:15:01-05:00,26.39,0.00,100,0,BAC PRB


In [5]:
ymd_dir()

(1000, 5)

In [16]:
df = pd.read_hdf('D://valkyrie//data_bak//AGNCP.h5')

In [61]:
stock_list = ['AGNCM']
date = '20201104'

In [100]:
zz = gen_marked_md(date, stock_list, '1s')

In [99]:
def gen_marked_md(date, stock_list, trade_marking_offset, 
                  output_path = 'D://valkyrie//data//marked_md',
                  src_path = 'D://valkyrie//data//hist_ib_raw_md'):
    os.makedirs(output_path + ymd_dir(date), exist_ok=True)
    src_path = 'D://valkyrie//data//hist_ib_raw_md' + ymd_dir(date)
        
    df_re = []
    for stk in stock_list:
        df_bidask = pd.read_hdf(src_path + f'//bid_ask//{stk}.h5')
        df_trades = pd.read_hdf(src_path + f'//trades//{stk}.h5')
        df_trades.index = df_trades.index - pd.Timedelta(trade_marking_offset)
        df_bidask = df_bidask.query('0 < priceBid < priceAsk ').copy()
        df_merged = pd.concat([df_bidask, df_trades]).sort_index()
        
        for c in 'priceBid priceAsk sizeBid sizeAsk'.split():
            df_merged[c] = df_merged[c].ffill()
        df_merged.rename({'priceBid' : 'bid',
                          'priceAsk' : 'ask',
                          'sizeBid'  : 'bz',
                          'sizeAsk'  : 'az',
                          'price'    : 'trd_px',
                          'size'     : 'trd_sz',
                          'exchange' : 'exch'
                         },axis=1, inplace=True)
        
        df_merged['mid']   = df_merged.eval('0.5 * (bid + ask)')
        df_merged['sprd']  = df_merged.eval('0.5 * (ask - bid)')
        df_merged['bs']    = df_merged.eval('100*(trd_px - mid)').clip(-1,1)
        df_merged['offset']=df_merged.eval('0.5*abs(bs)*((1+bs)*(ask-trd_px) + (1-bs)*(trd_px- bid))').clip(0,np.inf)        
        df_merged['date'] = index2dt(df_merged)        
        return df_merged
    
    df_re.append(df_merged)
    return pd.concat(df_re)

In [45]:
def analyze_pnl_and_volume(df, exchs = ['NYSE','ISLAND']):
    df_ex_trades = df_trades.query('exch != "FINRA"').copy()
    if exchs:
        df_ex_trades = df_ex_trades.query('exch in @exchs')

    df_re = df_ex_trades.query('sprd > 0.05 and offset <=0.02').groupby(['date','exch'])['trd_sz'].agg(['count','sum'])    
    return df_re, df_re.groupby('exch').agg(['median','mean','std','min','max'])